In [27]:
from __future__ import print_function
import keras
from keras.layers import Input, Conv2D, Dropout, MaxPooling2D, Flatten, Dense
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils import to_categorical
from keras import optimizers
from keras.models import model_from_json
import scipy.linalg
from sklearn.cluster import k_means
from sklearn.cluster import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import csv
import numpy as np
import pandas as pd
import random
import os
import copy


modeldir = "data/models/"
num_classes = 5
#MODIFY or ADD from cmd
coarse_categories = 3
fine_categories = num_classes

batch_size = 128
imsize = 128

# input image dimensions
img_x, img_y = imsize, imsize
input_shape = (img_x, img_y,3)
print(input_shape)

#ImageDataGenerator to generate batches of images
#add: zca_whitening = True,
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(imsize, imsize),
        batch_size=batch_size,
        class_mode='categorical')

#single classifier training (shared)
in_layer = Input(shape=input_shape, dtype='float32', name='main_input')

net = Conv2D(184, 3, strides=1, padding='same', activation='elu')(in_layer)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(184, 1, strides=1, padding='same', activation='elu')(net)
net = Conv2D(184, 2, strides=1, padding='same', activation='elu')(net)
net = Conv2D(284, 2, strides=1, padding='same', activation='elu')(net)
net = Conv2D(284, 2, strides=1, padding='same', activation='elu')(net)
net = Dropout(.2)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(284, 1, strides=1, padding='same', activation='elu')(net)
net = Conv2D(440, 2, strides=1, padding='same', activation='elu')(net)
net = Conv2D(440, 2, strides=1, padding='same', activation='elu')(net)
net = Dropout(.4)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(440, 3, strides=1, padding='same', activation='elu')(net)
net = Conv2D(568, 2, strides=1, padding='same', activation='elu')(net)
net = Conv2D(568, 2, strides=1, padding='same', activation='elu')(net)
net = Dropout(.5)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(568, 1, strides=1, padding='same', activation='elu')(net)
net = Conv2D(606, 2, strides=1, padding='same', activation='elu')(net)
net = Dropout(.6)(net)
net = MaxPooling2D((2, 2), padding='same')(net)

net = Flatten()(net)
net = Dense(606, activation='elu')(net)
net = Dense(fine_categories, activation='softmax')(net)


model = Model(inputs=in_layer,outputs=net)

model.load_weights(modeldir+"simplemodel_wt.h5")
print("Loaded simple model")

#fine-tuning for coarse classifier
net = Conv2D(568, 1, strides=1, padding='same', activation='elu')(model.layers[-8].output)
net = Conv2D(606, 2, strides=1, padding='same', activation='elu')(net)
net = Dropout(.6)(net)
net = MaxPooling2D((2, 2), padding='same')(net)

net = Flatten()(net)
net = Dense(606, activation='elu')(net)
out_coarse = Dense(coarse_categories, activation='softmax')(net)

model_c = Model(inputs=in_layer,outputs=out_coarse)

for i in range(len(model_c.layers)-1):
    model_c.layers[i].set_weights(model.layers[i].get_weights())

model_c.load_weights(modeldir+"coarsemodel_wt.h5")
print("Loaded coarse model")

#constructing fine classifiers
def fine_model():
    net = Conv2D(568, 1, strides=1, padding='same', activation='elu')(model.layers[-8].output)
    net = Conv2D(606, 2, strides=1, padding='same', activation='elu')(net)
    net = Dropout(.6)(net)
    net = MaxPooling2D((2, 2), padding='same')(net)

    net = Flatten()(net)
    net = Dense(606, activation='elu')(net)
    out_fine = Dense(fine_categories, activation='softmax')(net)
    model_fine = Model(inputs=in_layer,outputs=out_fine)
    

    for i in range(len(model_fine.layers)-1):
        model_fine.layers[i].set_weights(model.layers[i].get_weights())
    return model_fine

fine_models = {'models' : [{} for i in range(coarse_categories)], 'yhf' : [{} for i in range(coarse_categories)]}
for i in range(coarse_categories):
    model_i = fine_model()
    fine_models['models'][i] = model_i


# In[81]:


#training fine classifiers on corresponding data
for cat in range(coarse_categories):
    fine_models['models'][cat].load_weights(modeldir+str(cat)+"finemodel_wt.h5")
    
print("Loaded fine models")

#MODIFY to get accuracy
def get_error(t,p):
    #TODO add confidence score
    return accuracy_score(t,p)

#predicting
coarse_predictions=model_c.predict_generator(test_generator)
#predictions from #(coarse categories) fine classifiers
fine_predictions = []    #dim:  n_classes_coarse X n_images_predict X n_classes_fine
for c in range(coarse_categories):
    fine_predictions.append(fine_models['models'][c].predict_generator(test_generator))

print("Got predictions")

prediction_size = len(coarse_predictions)
predictions = []
for img in range(prediction_size):
    proba = [0]*fine_categories
    for finec in range(fine_categories):
        for coarsec in range(coarse_categories):
            proba[finec] += coarse_predictions[img][coarsec]*fine_predictions[coarsec][img][finec]
    predicted = np.argmax(proba)
    predictions.append(predicted)

truelabels = test_generator.classes

y_rand = np.random.choice(num_classes,len(truelabels))
e0 = get_error(truelabels,y_rand)
print("Error for random guess:",e0)

simple_pred= model.predict_generator(test_generator)
basic_p = []
for i in simple_pred:
    predicted = np.argmax(i)
    basic_p.append(predicted)

e1 = get_error(truelabels,basic_p)
print("Error for single classifier:",e1)

e2 = get_error(truelabels,predictions)
print("Error for HD-CNN classifier:",e2)


(128, 128, 3)
Found 15 images belonging to 5 classes.
Loaded simple model
Loaded coarse model
Loaded fine models
Got predictions
Error for random guess: 0.2
Error for single classifier: 0.2
Error for HD-CNN classifier: 0.2
